In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 32.1 MB/s 
     |████████████████████████████████| 166 kB 69.2 MB/s 
     |████████████████████████████████| 182 kB 64.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 68.7 MB/s 
     |████████████████████████████████| 162 kB 63.3 MB/s 
     |████████████████████████████████| 162 kB 56.9 MB/s 
     |████████████████████████████████| 158 kB 68.4 MB/s 
     |████████████████████████████████| 157 kB 72.3 MB/s 
     |████████████████████████████████| 157 kB 69.3 MB/s 
     |████████████████████████████████| 157 kB 70.7 MB/s 
     |████████████████████████████████| 157 kB 74.7 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 69.0 MB/s 
     |████████████████████████████████| 157 kB 68.5 MB/s 
     |████████████████████████████████| 156 kB 72.2 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [6]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [7]:
resnet50 = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg')
model_resnet50=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet50,
  #layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet50.trainable=False
model_resnet50.summary()

94765736/94765736 [==============================] - 7s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 32)                65568     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 23,653,445
Trainable params: 65,733
Non-trainable params: 23,587,712
________________________________________

In [8]:
model_resnet50.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [9]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet50 with hidden dense",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [10]:
%%time
epochs=40
history = model_resnet50.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.3770 - accuracy: 0.4347

wandb: Adding directory to artifact (/content/wandb/run-20221105_064701-2mvrsbbk/files/model-best)... Done. 0.9s


22/22 [==============================] - 314s 10s/step - loss: 1.3770 - accuracy: 0.4347 - val_loss: 1.3437 - val_accuracy: 0.5257
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.2710 - accuracy: 0.5440

wandb: Adding directory to artifact (/content/wandb/run-20221105_064701-2mvrsbbk/files/model-best)... Done. 0.9s


...

22/22 [==============================] - 26s 1s/step - loss: 0.8474 - accuracy: 0.6804 - val_loss: 0.9058 - val_accuracy: 0.6343
Epoch 35/40
22/22 [==============================] - ETA: 0s - loss: 0.8311 - accuracy: 0.6861

wandb: Adding directory to artifact (/content/wandb/run-20221105_064701-2mvrsbbk/files/model-best)... Done. 0.9s


22/22 [==============================] - 27s 1s/step - loss: 0.8311 - accuracy: 0.6861 - val_loss: 0.8922 - val_accuracy: 0.6743
Epoch 36/40
22/22 [==============================] - 4s 158ms/step - loss: 0.8340 - accuracy: 0.6932 - val_loss: 0.9166 - val_accuracy: 0.6229
Epoch 37/40
22/22 [==============================] - 4s 160ms/step - loss: 0.8527 - accuracy: 0.6889 - val_loss: 0.8935 - val_accuracy: 0.6514
Epoch 38/40
22/22 [==============================] - 4s 161ms/step - loss: 0.8424 - accuracy: 0.6733 - val_loss: 0.8996 - val_accuracy: 0.6114
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.8201 - accuracy: 0.6832

wandb: Adding directory to artifact (/content/wandb/run-20221105_064701-2mvrsbbk/files/model-best)... Done. 0.9s


22/22 [==============================] - 26s 1s/step - loss: 0.8201 - accuracy: 0.6832 - val_loss: 0.8687 - val_accuracy: 0.6743
Epoch 40/40
22/22 [==============================] - 4s 162ms/step - loss: 0.8124 - accuracy: 0.6861 - val_loss: 0.8735 - val_accuracy: 0.6400
CPU times: user 10min 35s, sys: 43.4 s, total: 11min 19s
Wall time: 19min 17s


In [11]:
wandb.finish()

accuracy,▁▄▄▄▅▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▇▇▇▇▇▇████▇██
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_accuracy,▁▂▂▃▄▄▃▃▄▂▂▃▂▂▂▂▂▃▃▃▃▅▄▄▆▆▄▅▄▅▆▅▆▆█▆▇▅█▆
val_loss,█▆▆▆▅▅▅▆▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▁▂▁▁▁▁
accuracy,0.68608
best_epoch,38
best_val_loss,0.86868
epoch,39
loss,0.81243
val_accuracy,0.64


In [12]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
